## Pipeline de um processo de Deep Learning implementado em PyTorch:

    1. Preparar os Dados
    2. Definir o Modelo
    3. Treinar o Modelo
    4. Avaliar o Modelo
    5. Usar o Modelo


## **MLP para classificação multiclasse**
**Dataset Iris (flores)**
- Dataset de imagens para previsão da espécie da flor dadas as medidas das flores
- 3 classes com 50 instâncias cada
- Classes: Iris Setosa, Iris Versicolour e Iris Virginica
- 5 atributos: 4 para dimensões numéricas e o 5º a classe da flor

## Imports

In [ ]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [ ]:
PATH = 'iris.csv'

device = torch.device("cpu")

EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 0.01

## 1. Preparar os Dados

In [ ]:
class CSVDataset(Dataset):
    def __init__(self, path):
        df = pd.read_csv(path, header=None)
        self.X = df.values[:, :-1]
        self.y = df.values[:, -1]
        self.X = self.X.astype('float32')
        self.y = LabelEncoder().fit_transform(self.y)
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    def get_splits(self, n_test=0.33):
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        return random_split(self, [train_size, test_size])
    
def prepare_data(path):
    dataset = CSVDataset(path)
    train, test = dataset.get_splits()
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    train_dl_all = DataLoader(train, batch_size=len(train), shuffle=False)
    test_dl_all = DataLoader(test, batch_size=len(test), shuffle=False)
    return train_dl, test_dl, train_dl_all, test_dl_all

train_dl, test_dl,  train_dl_all, test_dl_all = prepare_data(PATH)

## 1.1 Visualizar os Dados

In [ ]:
from IPython.display import display

def visualize_data(path):
    ...

def visualize_dataset(train_dl, test_dl):
    print(f"Quantidade de casos de Treino:{len(train_dl.dataset)}") 
    print(f"Quantidade de casos de Teste:{len(test_dl.dataset)}")
    x, y = next(iter(train_dl))casos
    print(f"Shape tensor batch casos treino, input: {x.shape}, output: {y.shape}")
    x, y = next(iter(test_dl))  
    print(f"Shape tensor batch casos test, input: {x.shape}, output: {y.shape}")
    print(y)

visualize_data(PATH)
visualize_dataset(train_dl, test_dl)

## 1.2 Verificar balanceamento do dataset

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def visualize_holdout_balance(y_train, y_test):
    ...

    sns.set_style('whitegrid')
    casos_treino=len(y_train) 
    casos_test=len(y_test)
    
    Iris_setosa_Train=np.count_nonzero(y_train == 0)
    Iris_versicolor_Train = np.count_nonzero(y_train == 1)
    Iris_virginica_Train = np.count_nonzero(y_train == 2)
    Iris_setosa_Test=np.count_nonzero(y_test == 0)
    Iris_versicolor_Test = np.count_nonzero(y_test == 1)
    Iris_virginica_Test = np.count_nonzero(y_test == 2)
    
    print("casos_treino:",casos_treino)
    print("Iris-setosa_Train: ", Iris_setosa_Train)
    print("Iris-versicolor_Train: ", Iris_versicolor_Train) 
    print("Iris-virginica_Train: ", Iris_virginica_Train)
    
    print("casos_test:",casos_test)
    print("Iris-setosa_Test: ", Iris_setosa_Test)
    print("Iris-versicolor_Test: ", Iris_versicolor_Test)
    print("Iris-virginica_Test: ", Iris_virginica_Test)

    grafico=sns.barplot(x=['Iris-setosa_Train','Iris-versicolor_Train', 'Iris-virginica_Train', 'Iris-setosa_Test', 'Iris-versicolor_Test', 'Iris-virginica_Test'], 
                        y=[Iris_setosa_Train, Iris_versicolor_Train, Iris_virginica_Train, Iris_setosa_Test, Iris_versicolor_Test, Iris_virginica_Test])
    grafico.set_title('Data balance ')
    plt.xticks(rotation=70)
    plt.tight_layout()
    plt.show() 
    
visualize_holdout_balance(train_dl_all, test_dl_all)

## 2. Definir o Modelo

In [ ]:
from torchinfo import summary

class MLP(Module):
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        ...
 
    def forward(self, X):
        X = self.hidden1(X)
        X = self.act1(X)
        X = self.hidden2(X)
        X = self.act2(X)
        X = self.hidden3(X)
        X = self.act3(X)
        return X

...
print(summary(model, input_size=(BATCH_SIZE, 4), verbose=0))
model.to(device)

## 3. Treinar o Modelo

In [ ]:
from livelossplot import PlotLosses

def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    return acc

def train_model(train_dl, model):
    liveloss = PlotLosses()
    criterion = CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
    for epoch in range(EPOCHS):
        logs = {}
        epoch_loss  = 0
        epoch_acc  = 0
        for i, (inputs, labels) in enumerate(train_dl):
            optimizer.zero_grad()
            yprev = model(inputs)
            loss = criterion(yprev, labels)
            ...
            loss.backward()
            optimizer.step()
            ...
        
        print(f'Epoch {epoch:03}: | Loss: {epoch_loss/len(train_dl):.5f} | Acc: {epoch_acc/len(train_dl):.3f}')      
        logs['loss'] = epoch_loss
        logs['accuracy'] = epoch_acc/len(train_dl)
        ...

train_model(train_dl, model)

## 4. Avaliar o Modelo

In [ ]:
def evaluate_model(test_dl, model):
    predictions = list()
    actual_values = list()
    for i, (inputs, labels) in enumerate(test_dl):
        yprev = model(inputs)
        yprev = yprev.detach().numpy()
        actual = labels.numpy()
        yprev = np.argmax(yprev, axis=1)
        actual = actual.reshape((len(actual), 1))
        yprev = yprev.reshape((len(yprev), 1))
        predictions.append(yprev)
        actual_values.append(actual)
        break
    predictions, actual_values = np.vstack(predictions), np.vstack(actual_values)
    return predictions, actual_values
 
def display_confusion_matrix(cm):
    plt.figure(figsize = (16,8))
    sns.heatmap(cm,annot=True,xticklabels=['Iris-setosa','Iris-versicolor','Iris-virginica'],yticklabels=['Iris-setosa','Iris-versicolor','Iris-virginica'], annot_kws={"size": 12}, fmt='g', linewidths=.5)

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show() 
    
predictions, actual_values = evaluate_model(test_dl, model)

acertou=0
falhou = 0
for r,p in zip(actual_values, predictions):
    print(f'real:{r} previsão:{p}') 
    if r==p: acertou+=1  
    else: falhou+=1

acc = accuracy_score(actual_values, predictions)
print(f'Accuracy: {acc:0.3f}\n')
print(f'acertou:{acertou} falhou:{falhou}')

...

## 5. Usar o Modelo

In [ ]:
def predict(row, model):
    row = Tensor([row])
    yprev = model(row)
    yprev = yprev.detach().numpy()
    return yprev

...
yprev = predict(row, model)
print('Predicted: %s (class=%d)' % (yprev, np.argmax(yprev)))